In [16]:
import vk_api 
import pandas as pd
from datetime import datetime, timedelta

access_token = '9f0e49589f0e49589f0e4958ca9c18fc0499f0e9f0e4958fa8bc775efed21e63550ef79'
vk_session = vk_api.VkApi(token=access_token) 
vk = vk_session.get_api()

group_link = 'https://vk.com/te_ekb'
group_info = vk.utils.resolveScreenName(screen_name=group_link.replace('https://vk.com/', ''))
owner_id = -group_info['object_id'] # ID группы

current_time = datetime.now()
half_year_ago = current_time - timedelta(days=180)

data = []

offset = 0
count = 100

while True:
    posts = vk.wall.get(owner_id=owner_id, offset=offset, count=count)['items']
    
    for post in posts:
        post_time = post['date']
        post_text = post['text']
        
        if half_year_ago.timestamp() <= post_time <= current_time.timestamp():
            post_time_str = datetime.fromtimestamp(post_time).strftime('%d-%m-%Y')

            # Get comments for the current post
            comments = vk.wall.getComments(owner_id=owner_id, post_id=post['id'], count=10)
            comments_count = comments['count']
            
            comments_text = []  # Empty list to store comment text
            
            for comment in comments['items']:
                comment_text = comment['text']
                comments_text.append(comment_text)

            data.append({
                'Время публикации': post_time_str,
                'Текст поста': post_text,
                'Количество комментариев': comments_count,
                'Текст комментариев': comments_text
            })
    
    offset += count
    
    if len(posts) < count or post_time < half_year_ago.timestamp():
        break
    
df = pd.DataFrame(data)
df.to_excel('posts_data_with_comments.xlsx', index=False)